In [ ]:
# default_exp data

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Data

> This model contains all the necessary functionality for managing data.

In [ ]:
# export
from pathlib import Path
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers, processors

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# hide
from ds4se.mgmnt.prep.i import jsonl_list_to_dataframe, get_dfs

path = Path('/home/nathan/Downloads/')
df_trn, df_val, df_tst = get_dfs(path/"java/final/jsonl")

sample = 0.01
df_trn = df_trn.sample(frac = sample)
df_val = df_val.sample(frac = sample)
df_tst = df_tst.sample(frac = sample)
df_trn.head()

,code,docstring
22198,protected Object[] validateArguments(Method me...,Validates the array of {@link Object arguments...
5223,"@SuppressWarnings(""rawtypes"")\n\tpublic void s...",Sets the class loader to be used on serializat...
29127,public ServiceFuture<List<BackupEngineBaseReso...,The backup management servers registered to a ...
5079,private void ensureMetaCharsetElement() {\n ...,Ensures a meta charset (html) or xml declarati...
11797,public InstallationJob getJob(Plugin plugin) {...,Returns latest install/upgrade job for the giv...


In [ ]:
# hide
len(df_trn), len(df_val), len(df_tst)

(4545, 153, 269)

In [ ]:
# export
# TODO: add special tokens and test case
def train_tokenizer(df, n = None, vocab_sz = 20_000, min_freq = 2, output = None):
    """
    Train a ByteLevel BPE tokenizer on a given pandas dataframe. Code adapted from https://github.com/huggingface/tokenizers/tree/master/bindings/python.

    :param df: the pandas dataframe containing each method to have the tokenizer train on
    :param n: the number of methods to evaluate. If none, the entire dataframe will be used
    :param vocab_sz: the maximum vocabulary size of the trained tokenizer
    :param min_freq: the minimum frequency a token has to occur to be considered
    :returns: returns a trained ByteLevel BPE tokenizer
    """
    if n is None: n = len(df)

    # create tmp file to store df contents for training tokenizer
    tmp_path = Path('/tmp')
    tmp_path.mkdir(parents = True, exist_ok = True)
    with open(tmp_path/'tmp_tokenize.txt', 'w') as f:
        f.write('\n'.join(df.code.values[:n]))

    # initialize a tokenizer
    tokenizer = Tokenizer(models.BPE())

    # customize pre-tokenization and decoding
    tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space = True)
    tokenizer.decoder = decoders.ByteLevel()
    tokenizer.post_processor = processors.ByteLevel(trim_offsets = True)

    # train tokenizer with data in tmp file
    trainer = trainers.BpeTrainer(vocab_size = vocab_sz, min_frequency = min_freq)
    tokenizer.train(trainer, [str(tmp_path/'tmp_tokenize.txt')])

    # save tokenizer if output path given
    if output is not None:
        tokenizer.save(output, pretty = True)
    
    return tokenizer

In [ ]:
# hide
tokenizer = train_tokenizer(df_trn)

Encoding(num_tokens=248, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
# hide
idx = 0
encoded = tokenizer.encode(df_trn.code.values[idx])
print(df_trn.code.values[idx])
print('=' * 100)
print(encoded.tokens)

protected Object[] validateArguments(Method method, Object... args) {
    Assert.notNull(method, &quot;Method cannot be null&quot;);

    Object[] arguments = nullSafeArray(args);

    int methodParameterCount = method.getParameterCount();

    assertThat(arguments.length)
      .throwing(newIllegalArgumentException(
        &quot;The number of arguments [%1$d] does not match the number of parameters [%2$d] for method [%3$s] in class [%4$s]&quot;,
        arguments.length, methodParameterCount, method.getName(), method.getDeclaringClass().getName()))
      .isEqualTo(methodParameterCount);

    Class&lt;?&gt;[] parameterTypes = method.getParameterTypes();

    int parameterIndex = 0;

    for (Object argument : arguments) {
      assertThat(argument)
        .throwing(newIllegalArgumentException(&quot;Argument [%1$s] is not assignable to parameter [%2$d] of type [%3$s]&quot;,
          argument, parameterIndex, parameterTypes[parameterIndex].getName()))
        .isAssignableTo(paramete